In [5]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp

exact_value = 0.77143

def an(n: int):
    return 24 / (7 * (n ** 2 + 8 * n + 15))


In [6]:
def count_correct_significant_digits(error, value):
    if error > value:
        return 0

    error_string = str(format(float(error), 'f'))

    counter = 0
    for digit_error in error_string:
        if digit_error.isdigit():
            err = float(digit_error)
            if err <= 1:
                counter += 1
            else:
                break
    return counter

In [7]:
def calculate_error(N: int, exact_value: float):
    partial_sum = 0
    for i in range(1, N):
        partial_sum += an(i)
    error = np.abs(exact_value - partial_sum)

    correct_numbers = count_correct_significant_digits(error, exact_value)
    return partial_sum, error, correct_numbers

In [8]:
M_array = []
N_array = []
for N in [10, 10 ** 2, 10 ** 3, 10 ** 4, 10 ** 5]:
    data = calculate_error(N, exact_value)
    M_array.append(data[2])
    N_array.append(N)
    print(f"S({N}) = {data[0]}, d({N}) = {data[1]}, M = {data[2]}")

In [9]:
plt.bar(M_array, (M_array + [0])[0:-1])
plt.xticks((M_array + [0])[0:-1], N_array)
plt.title("Гистограмма количества верных цифр к размеру частичной суммы")

In [12]:
delta = 0.01
a1, a2, a3, a4, a5 = sp.symbols("a1 a2 a3 a4 a5")
matrix_a = sp.Matrix([
    [a1, 1, a5],
    [5, a2, 15],
    [a4, 5, a3]
])
print(f"Exact determinant: {matrix_a.det().subs([(a1, 3), (a2, 3), (a3, 40), (a4, 11), (a5, 13)])}")

exact_determinant = matrix_a.det().subs([(a1, 3), (a2, 3), (a3, 40), (a4, 11), (a5, 13)])

for case in [1 + delta, 1 - delta]:
    for aii, i in zip([a1, a2, a3, a4, a5], [(0, 0), (1, 1), (2, 2), (0, 2), (2, 0)]):
        new_matrix = matrix_a.copy()
        new_matrix[i[0], i[1]] = new_matrix[i[0], i[1]] * case
        determinant = new_matrix.det()

        derivative = sp.diff(determinant, aii)

        relative_error = ((sp.Abs(derivative) * sp.Abs(aii)) / sp.Abs(determinant))  * delta
        absolute_error = relative_error * sp.Abs(aii)

        significant_digits = count_correct_significant_digits(absolute_error.subs([(a1, 3), (a2, 3), (a3, 40), (a4, 11), (a5, 13)]), abs(exact_determinant))

        print(f"{aii}: {case}; Relative error = {relative_error.subs([(a1, 3), (a2, 3), (a3, 40), (a4, 11), (a5, 13)])}, Absolute error = {absolute_error.subs([(a1, 3), (a2, 3), (a3, 40), (a4, 11), (a5, 13)])}, True numbers: {significant_digits}")

In [2]:
def machine_zero(type):
    k = 0
    value = type(1)
    while value != 0:
        value = type(value / 2)
        k += 1
    print(f"{type.__name__}: Машинный ноль = 2^-{k}")


def machine_infinity(type):
    k = 0
    value = type(1)
    while value != np.inf:
        value = type(value * 2)
        k += 1
    print(f"{type.__name__}: Машинная бесконечность = 2^{k}")


def machine_epsilon(type):
    k = 0
    value = type(1)
    while type(1.) + value > type(1.):
        value = type(value / 2)
        k += 1
    print(f"{type.__name__}: Машинное эпсилон = 2^-{k}")


for my_type in [np.single, np.double, np.longdouble]:
    machine_zero(my_type)
    machine_infinity(my_type)
    machine_epsilon(my_type)
    print()

In [3]:
from itertools import product

def inverse_matrix_check(matrix):
    print(f'Без погрешности: detA = {np.linalg.det(matrix)}\n')


def inverse_matrix_check_error(matrix, delta):
    determinants = []
    for j in list(product([-1, 1], repeat=9)):
        determinants.append(
            np.linalg.det(
                matrix * (1 + delta * np.array(j).reshape(3, 3))
            )
        )

    min_determinant, max_determinant = np.min(determinants), np.max(determinants)
    print(f'Минимальный определитель = {min_determinant}')
    print(f'Максимальный определитель = {max_determinant}')

    if min_determinant < 0 < max_determinant:
        print(f"С погрешностью {delta}: определитель может быть 0\n")
    else:
        print(f"С погрешностью {delta}: определитель не может быть 0\n")


matrix = np.array([
    [3, 1, 13],
    [13.4, 11.4, 23],
    [5, 3, 15]
])
inverse_matrix_check(matrix)
inverse_matrix_check_error(matrix, 0.05)
inverse_matrix_check_error(matrix, 0.1)